In [19]:
import argparse 
import ast

import cf_xarray
#import cftime
import geocat.comp as gcomp
#import holoviews as hv
#import hvplot
#import hvplot.xarray
#import intake
import numpy as np
#import pop_tools
import xarray as xr
import xesmf as xe

#from distributed import Client
#from ncar_jobqueue import NCARCluster
#from pop_tools.grid import _compute_corners

import logging 
import netCDF4 as nc
from numba import jit 

## Questions: 
Do certain variables still need to be reversed?  
Is surface geopotential and lowest layer geopotential treated the same here? 
Does geocat's interp function work across time? 
Is it better to build an atomic function and use appy_ufunc or build in vectorization? 
If you use numpy functions on xarray dataarrays, does xarray intercept the function to correct the dimensions? 

TO DO:  
Implement weight reuse  
Implement pressure levels   
Fix wrong standard name for SST  
Add hooks for user-defined reference pressure?  
Add hook to keep metadata  
Fix up METGRID default value  
Implement unit converter (pint) or unit checker  
NOTES:   
Created a branch for a version that works on cf-compliant data  
 


In [31]:
#Command line option handling ----------------------------------------------------------------------------------
parser = argparse.ArgumentParser()  
logging.basicConfig(level=logging.DEBUG)
current_log_level = logging.getLogger().getEffectiveLevel() 

parser.add_argument('CASE',type=str, help='One of the following IPCC Climate Scenarios: 20THC/RCP85/RCP60/RCP45')
parser.add_argument('--o',type=str,help='Output directory path')
parser.add_argument('--mode','-m',type=str,help='Set logging mode: DEBUG/INFO/WARNING/ERROR/CRITICAL')
parser.add_argument('--plev',type=str, help="File name of desired output pressure levels")
parser.add_argument('--weights',type=str, help="File name if reusing regridding weights")

if current_log_level != 10: 
    args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--o O] [--mode MODE] [--plev PLEV]
                             [--weights WEIGHTS]
                             CASE
ipykernel_launcher.py: error: the following arguments are required: CASE


SystemExit: 2

/glade/work/wukenton/conda-envs/python_correct/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
#File Handling ----------------------------------------------------------------------------------

logging.info("Opening data files...")

in_ta = xr.open_dataset("atmos_ta.nc")         # 6-hourly 3-d T
in_ua = xr.open_dataset("atmos_ua.nc")         # 6-hourly 3-d U
in_va = xr.open_dataset("atmos_va.nc")         # 6-hourly 3-d V
in_hus = xr.open_dataset("atmos_hus.nc")       # 6-hourly 3-d Q
in_ps = xr.open_dataset("atmos_ps.nc")         # 6-hourly surface pressure
in_zsfc = xr.open_dataset("atmos_zsfc.nc")     # static surface geopotential
in_lmask = xr.open_dataset("atmos_lmask.nc")   # static land mask
in_snw = xr.open_dataset("atmos_snw_1.nc")     # monthly SWE
in_mrlsl = xr.open_dataset("atmos_mrlsl_1.nc") # monthly soil moisture
in_ts = xr.open_dataset("atmos_ts_1.nc")       # monthly skin temp
in_tsl = xr.open_dataset("atmos_tsl_1.nc")     # monthly soil temp
in_tos = xr.open_dataset("atmos_tos_1.nc")     # daily SST on pop grid (gaussian)
in_sic = xr.open_dataset("atmos_sic_1.nc")     # daily SEAICE % on POP grid (gaussian)

In [ ]:
#Regrid SST and SEA ICE fields to CESM Atmospheric Domain ----------------------------------------------------------------------------------

logging.info('Converting Parallel Ocean Program data to coordinate system of atmospheric grid...')

SST = in_tos.cf['surface_temperature']
#Create a mask (not needed for interpolating to atmospheric grid, but just in case there are missing values)
#NOTE THAT THIS CF REFERENCE IS WRONG. SST IS THE CORRECT STANDARD NAME WHICH NEEDS TO BE CORRECTED IN THE DATA
in_tos["mask"] = ~SST.cf.isel(time=0).isnull()

#Regrids SST grid to whatever the atmospheric grid is automatically
regrid = xe.Regridder(in_tos, in_ta, method = 'bilinear', periodic=True, unmapped_to_nan=True)
regrid.to_netcdf('weights_gx1v6_latlon.nc') #write out weights for reuse 

regridded_SST = regrid(in_tos)

if current_log_level == 10: 
    print(regridded_SST)
#regridded_SST.to_netcdf('python_regrid.nc')
#
#use some sort of broadcasting or view here to clone to a 6-hrly variable

INFO:root:Converting Parallel Ocean Program data to coordinate system of atmospheric grid...
/glade/work/wukenton/conda-envs/python_correct/lib/python3.9/site-packages/cf_xarray/accessor.py:1772: UserWarning: Variables {'areacello'} not found in object but are referred to in the CF attributes.
  warnings.warn(
/glade/work/wukenton/conda-envs/python_correct/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1037)
           2	LOAD_GLOBAL(arg=0, lineno=1053)
           4	LOAD_ATTR(arg=1, lineno=1053)
           6	LOAD_FAST(arg=3, lineno=1053)
           8	LOAD_DEREF(arg=0, lineno=1053)
          10	LOAD_CONST(arg=1, lineno=1053)
          12	CALL_FUNCTION_KW(arg=2, lineno=1053)
          14	STORE_FAST(arg=4, lineno=1053)
          16	LOAD_CONST(arg=2, lineno=1054)
      

<xarray.Dataset>
Dimensions:   (vertices: 4, lat: 192, lon: 288, time: 92)
Coordinates:
  * time      (time) object 2006-10-01 12:00:00 ... 2006-12-31 12:00:00
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: vertices
Data variables:
    lon_bnds  (vertices, lat, lon) float32 nan nan nan nan ... 248.7 248.7 248.7
    lat_bnds  (vertices, lat, lon) float32 nan nan nan nan ... 89.65 89.65 89.65
    tos       (time, lat, lon) float32 nan nan nan nan ... 271.3 271.3 271.3
    mask      (lat, lon) bool True True True True True ... True True True True
Attributes:
    regrid_method:  bilinear


In [3]:
#Prepare Variables for Interpolation ----------------------------------------------------------------------------------

hyam = in_ta.cf['hyam'] 
hybm = in_ta.cf['hybm']
hyai = in_ta.cf['hyai']
hybi = in_ta.cf['hybi']

surf_pressure = in_ps.cf['PS']

phi_surf = in_zsfc['PHIS']
phi_surf.coords['lat'] = surf_pressure.coords['lat']
phi_surf.coords['lon'] = surf_pressure.coords['lon']
temp = in_ta["T"]    


Check that this is correct - it seems like the coordinate system is somewhat reversed.

In [20]:
%%timeit -r 10

#need assertion that missing values are correct 
@jit 
def pressure_on_hybrid_ccm(pressure_surf : xr.DataArray, hyam, hybm, ref_pressure : xr.DataArray =  100000): 
    
    assert 'positive' in hyam.cf['Z'].attrs; "CF Standard requires that non-pressure data has an assigned direction in its vertical coordinate"
    assert 'positive' in hybm.cf['Z'].attrs; "CF Standard requires that non-pressure data has an assigned direction in its vertical coordinate"
    
    #since xarray defaults to nan's for missing values, we can rely on xarray to take care of missing values
    return hyam*ref_pressure + hybm*pressure_surf 
    

P_hybrid = pressure_on_hybrid_ccm(surf_pressure,hyam,hybm)

<magic-timeit>:3: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
<magic-timeit>:2: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "pressure_on_hybrid_ccm" failed type inference due to: non-precise type pyobject
During: typing of argument at <magic-timeit> (2)

File "<magic-timeit>", line 2:
<source missing, REPL/exec in use?>

/glade/work/wukenton/conda-envs/python_correct/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "pressure_on_hybrid_ccm" was compiled in object mode without forceobj=True.

File "<magic-timeit>", line 2:
<source missing, REPL/exec in use?>

  warnings.warn(errors

4.6 s ± 297 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [ ]:

ncl_P_hybrid = xr.open_dataarray('ncl_P_hybrid.nc')
ncl_P_hybrid.coords['time'] = P_hybrid.coords['time']
ncl_P_hybrid.coords['lev'] = P_hybrid.coords['lev']
ncl_P_hybrid.coords['lat'] = P_hybrid.coords['lat']
ncl_P_hybrid.coords['lon'] = P_hybrid.coords['lon']
P_hybrid - ncl_P_hybrid

In [36]:


def psl_ecmwf(temp_bottom: xr.DataArray, phi_surf: xr.DataArray, pressure_surf: xr.DataArray, pressure_bot: xr.DataArray): 
    #Based on the NCAR Technical Note "Vertical Interpolation and Truncation of Model-Coordinate Data"
    #By Trenberth, Berry, Buja; Dec 1993 
    #temp_surf = T_*, temp_bottom = T_NL, pressure_surf = p_s, pressure_bot = p_NL, phi_surf = \phi_s

    LAPSE_RATE = 0.0065     #Kelvin per meter
    GRAV_CONST = 9.80616    #Meters per second per second
    SPEC_GAS_CONST = 287.04 #Joules per kilogram per Kelvin

    ALPHA_0 = LAPSE_RATE*SPEC_GAS_CONST/GRAV_CONST
    
    phi_surf_expanded = phi_surf.broadcast_like(pressure_surf)
    temp_surf = temp_bottom*(1 + ALPHA_0*(pressure_surf/pressure_bot - 1)) #3b.5
    temp_bot_lapse = temp_surf + LAPSE_RATE*phi_surf/GRAV_CONST #denoted T_0 in doc, 3b.13

    is_near_zero = np.full_like(pressure_surf,False,dtype=bool)
    is_hot_low = np.full_like(pressure_surf,False,dtype=bool)
    is_hot_high = np.full_like(pressure_surf,False,dtype=bool)
    is_mild_high = np.full_like(pressure_surf,False,dtype=bool)
    is_cold_low = np.full_like(pressure_surf,False,dtype=bool)
    is_cold_high = np.full_like(pressure_surf,False,dtype=bool)

    is_near_zero =  np.absolute(phi_surf_expanded/GRAV_CONST) < 1e-4 #this has precedence over the others 
    is_hot_low = np.logical_and(temp_surf >= 255,temp_bot_lapse <= 290.5, where = ~is_near_zero, out = is_hot_low)  
    is_hot_high = np.logical_and(temp_surf > 290.5, temp_bot_lapse > 290.5, where = ~is_near_zero, out =  is_hot_high) 
    is_mild_high = np.logical_and(temp_surf >=255, np.logical_and(temp_surf <= 290.5, temp_bot_lapse > 290.5, 
                                                where = ~is_near_zero, out = is_mild_high), where = ~is_near_zero, out= is_mild_high)  
    np.logical_and(temp_surf < 255, temp_bot_lapse <= 290.5, where = ~is_near_zero, out = is_cold_low) 
    test = np.logical_and(temp_surf < 255, temp_bot_lapse > 290.5, where = ~is_near_zero, out = is_cold_high)  

    #create vectorized functions 
    def psl_hot_low(ps,phi_s,T_star): 
        combo_term = ALPHA_0*phi_s/SPEC_GAS_CONST/T_star 
        return ps*np.exp(combo_term/ALPHA_0*(1-1/2*(combo_term)+1/3*(combo_term)**2))
    def psl_hot_high(ps,phi_s,T_star): 
        T_star_modified = 1/2*(290.5+T_star) 
        return ps*np.exp(phi_s/SPEC_GAS_CONST/T_star_modified)
    def psl_mild_high(ps,phi_s,T_star):
        combo_term = 290.5-T_star
        return ps*np.exp(phi_s/SPEC_GAS_CONST/T_star*(1-1/2*(combo_term)+1/3*(combo_term)**2))
    def psl_cold_low(ps,phi_s,T_star): 
        T_star_modified = 1/2*(255+T_star)
        combo_term = ALPHA_0*phi_s/SPEC_GAS_CONST/T_star_modified 
        return ps*np.exp(combo_term/ALPHA_0*(1-1/2*(combo_term)+1/3*(combo_term)**2))
    #Check this function - it may be wrong in the ncl code! 
    def psl_cold_high(ps,phi_s,T_star): 
        alpha = SPEC_GAS_CONST/phi_s*(290.5-T_star)
        T_star_modified = 1/2*(255+T_star)
        combo_term = alpha*phi_s/SPEC_GAS_CONST/T_star_modified 
        return ps*np.exp(combo_term/alpha*(1-1/2*(combo_term)+1/3*(combo_term)**2))    
    def psl_near_zero(ps,phi_s,T_star): return ps

    formulas =  [psl_near_zero,  psl_hot_low,    psl_hot_high,   psl_mild_high,  psl_cold_low,   psl_cold_high]
    cases =     [is_near_zero,   is_hot_low,     is_hot_high,    is_mild_high,   is_cold_low,    is_cold_high]

    try: 
        assert np.logical_xor.reduce(cases,axis=0).all()
    except AssertionError: 
        assert np.any(cases,axis = 0).all(), "Underlap in cases"
        assert False, "Overlap in cases"

    psl = np.full_like(pressure_surf,np.nan, dtype=)
    for where_case, formula in zip(cases,formulas): 
            psl[where_case] = formula(pressure_surf.to_numpy()[where_case],phi_surf_expanded.to_numpy()[where_case],temp_surf.to_numpy()[where_case])
    psl = pressure_surf.copy(data=psl)
    return psl 

test_pressure = psl_ecmwf(temp.isel(lev=-1,time=0), phi_surf, surf_pressure.isel(time=0), P)


SyntaxError: invalid syntax (4154432559.py, line 62)

In [19]:
%%timeit

test_pressure.to_netcdf('python_psl.nc')

default_levs = np.array([1000.0, 975.0, 950.0, 925.0, 900.0, 850.0, 800.0, 750.0, 700.0, 650.0, 600.0, 550.0, 500.0, \
             450.0, 400.0, 350.0, 300.0, 250.0, 200.0, 150.0, 100.0, 70.0, 50.0, 30.0, 20.0, 10.0 ])


7.56 ms ± 982 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
#Interpolate to Pressure Coordinates ----------------------------------------------------------------------------------
#logging.info("Interpolating variables to pressure coordinates...")

#if current_log_level == 10: print(temp); print(surf_pressure); print(in_zsfc['PHIS']); 
hyam = in_ta.cf['hyam'] 
hybm = in_ta.cf['hybm']
hyai = in_ta.cf['hyai']
hybi = in_ta.cf['hybi']
default_levs = np.array([1000.0, 975.0, 950.0, 925.0, 900.0, 850.0, 800.0, 750.0, 700.0, 650.0, 600.0, 550.0, 500.0, \
             450.0, 400.0, 350.0, 300.0, 250.0, 200.0, 150.0, 100.0, 70.0, 50.0, 30.0, 20.0, 10.0 ])


surf_pressure = in_ps.cf['PS']
phi_surf = in_zsfc['PHIS']
temp = in_ta["T"]    
fixed_phi_sfc = in_zsfc['PHIS']  
fixed_phi_sfc.coords['lat'] = surf_pressure.coords['lat']
fixed_phi_sfc.coords['lon'] = surf_pressure.coords['lon'] 

temp_interp = gcomp.interpolation.interp_hybrid_to_pressure(temp.isel(time=0),surf_pressure.isel(time=0),hyam,hybm, 
                                                            new_levels=default_levs, 
                                                            lev_dim = 'lev', 
                                                            method='log',
                                                            extrapolate=True,
                                                            variable='temperature',
                                                            t_bot=temp.isel(lev=-1,time=0),
                                                            phi_sfc=in_zsfc['PHIS'])

ValueError: cannot align objects with join='exact' where index/labels/sizes are not equal along these coordinates (dimensions): 'lat' ('lat',)